# Load Horizons Data into KeplerDB

In [1]:
# Core
import numpy as np
import pandas as pd

# Utility
import re
import glob
import time

# Database
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
# MSE imports
import kepler_sieve
from horizons_files import hrzn_txt2csv, hrzn_csv2df, hrzn_df2db, hrzn_txt2db, hrzn_csv2db, hrzn_load
from asteroid_element import load_ast_elt, load_data_impl, convert_data
from astro_utils import anomaly_M2E, anomaly_E2f
import db_config

## Load all the Horizons CSV Files into JPL.HorizonsImport

In [3]:
# hrzn_load()

## Testing - Load CSV, Display as DataFrame

In [4]:
fname_txt = '../data/jpl/horizons/planets/daily/010_sun.txt'
# fname_txt = '../data/jpl/horizons/planets/daily/001_mercury_barycenter.txt'
# fname_txt = '../data/jpl/horizons/moons/daily/301_moon.txt'
# fname_txt = '../data/jpl/horizons/asteroids/weekly/ast_0001.txt'

In [5]:
fname_csv = hrzn_txt2csv(fname_txt)

In [6]:
fname_csv

'/ssd1/tmp/mysql/jpl/010_sun.csv'

In [7]:
df = hrzn_csv2df(fname_csv)

In [8]:
pd.set_option('display.max_rows', 10)

In [10]:
df

,Num,Name,epoch,a,e,inc,Omega,omega,M,H,G,Ref,E,f,P,n,long,theta,row_num
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.40,0.12,JPL 47,1.425607,1.501306,1683.145658,0.003733,4.036516,4.187424,0
2,2,Pallas,59000.0,2.773841,0.229972,0.607949,3.019851,5.414053,2.530303,4.20,0.11,JPL 37,2.640731,2.742191,1687.410960,0.003724,4.681022,4.892910,1
3,3,Juno,59000.0,2.668285,0.256936,0.226736,2.964468,4.329572,2.189260,5.33,0.32,JPL 112,2.368662,2.535135,1592.013779,0.003947,3.200114,3.545989,2
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.00,0.32,JPL 35,1.760241,1.846768,1325.432768,0.004740,6.115656,0.006132,3
5,5,Astraea,59000.0,2.574037,0.190913,0.093679,2.470881,6.259596,0.311477,6.90,0.15,JPL 114,0.382785,0.461775,1508.414438,0.004165,2.758769,2.909067,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413590,1413590,6013 P-L,37203.0,2.280861,0.185919,0.104448,3.389951,3.411120,5.970031,17.14,0.15,JPL 5,5.900629,5.823982,1258.191236,0.004994,0.204732,0.058683,958719
1413591,1413591,6331 P-L,59000.0,2.334910,0.282919,0.141062,6.200251,0.091040,4.539829,18.50,0.15,JPL 8,4.282636,4.036234,1303.177715,0.004821,4.547935,4.044341,958720
1413592,1413592,6344 P-L,59000.0,2.817151,0.662446,0.081669,3.193653,4.099689,4.197056,20.40,0.15,JPL 17,3.794579,3.444418,1727.084616,0.003638,5.207212,4.454574,958721


## Testing - Populate Database from CSV

In [ ]:
connection_str = f'mysql+pymysql://{db_config.username}:{db_config.password}@{db_config.hostname}/JPL'

In [ ]:
engine = create_engine(connection_str)

In [ ]:
engine.table_names()

In [ ]:
# hrzn_txt2db(fname_txt)

## Load the Asteroid Orbital Elements from JPL

In [9]:
df = load_ast_elt()

In [11]:
df

,Num,Name,epoch,a,e,inc,Omega,omega,M,H,G,Ref,E,f,P,n,long,theta,row_num
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.40,0.12,JPL 47,1.425607,1.501306,1683.145658,0.003733,4.036516,4.187424,0
2,2,Pallas,59000.0,2.773841,0.229972,0.607949,3.019851,5.414053,2.530303,4.20,0.11,JPL 37,2.640731,2.742191,1687.410960,0.003724,4.681022,4.892910,1
3,3,Juno,59000.0,2.668285,0.256936,0.226736,2.964468,4.329572,2.189260,5.33,0.32,JPL 112,2.368662,2.535135,1592.013779,0.003947,3.200114,3.545989,2
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.00,0.32,JPL 35,1.760241,1.846768,1325.432768,0.004740,6.115656,0.006132,3
5,5,Astraea,59000.0,2.574037,0.190913,0.093679,2.470881,6.259596,0.311477,6.90,0.15,JPL 114,0.382785,0.461775,1508.414438,0.004165,2.758769,2.909067,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1413590,1413590,6013 P-L,37203.0,2.280861,0.185919,0.104448,3.389951,3.411120,5.970031,17.14,0.15,JPL 5,5.900629,5.823982,1258.191236,0.004994,0.204732,0.058683,958719
1413591,1413591,6331 P-L,59000.0,2.334910,0.282919,0.141062,6.200251,0.091040,4.539829,18.50,0.15,JPL 8,4.282636,4.036234,1303.177715,0.004821,4.547935,4.044341,958720
1413592,1413592,6344 P-L,59000.0,2.817151,0.662446,0.081669,3.193653,4.099689,4.197056,20.40,0.15,JPL 17,3.794579,3.444418,1727.084616,0.003638,5.207212,4.454574,958721
